<h1> COVID19 - Morocco</h1>
The COVID-19 pandemic in Morocco is part of the worldwide pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The virus was confirmed to have spread to Morocco on 2 March 2020, when the first case COVID-19 case was confirmed in Casablanca. This notebook aims to analyze and explore COVID-19 pandemic in Morocco through data analysis and projections.

This is a work **in progress**. Please **upvote** the kernel to encourage this work. 

**<h3>Visualization:**
- Total Cumulative COVID-19 Cases in Morocco
- Total Evolution of COVID-19 Active Cases
- Pie Chart of COVID-19 Cases per Region
- Total Evolution of COVID-19 Cases per Region
- Plots of Evolution of COVID-19 Cases per Region
- Forecasting of COVID-19 Daily Cases

**<h3>Resources:**
- [Moroccan Ministry of Health](http://www.covidmaroc.ma/pages/Accueil.aspx)
- [COVID-19 in Morocco - Kaggle Dataset](https://www.kaggle.com/abdelghanibelgaid/covid19-morocco)


Contact me if you have a contribution or a suggestion. Your feedback is much appreciated!

In [ ]:
import math, time, datetime
from datetime import date
import numpy as np, pandas as pd

from fbprophet import Prophet
from sklearn.metrics import mean_absolute_error

from fbprophet.plot import plot_plotly, plot_components_plotly
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import squarify

In [ ]:
data = pd.read_csv("../input/covid19-morocco/covid19-morocco.csv")
data["Date"] = pd.to_datetime(data['Date'], dayfirst=True)
data = data[data["Date"] <= pd.Timestamp(date.today())]
active = data['Confirmed'] - data['Recovered'] - data['Deaths']
data['Active'] = active
total_Confirmed=data['Confirmed']
# data.info()

In [ ]:
data.tail()

In [ ]:
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
temp = data[['Date','Deaths', 'Recovered', 'Active']].tail(1)
temp = temp.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Recovered'])
fig = px.treemap(temp, path=["variable"], values="value", height=225, 
                 color_discrete_sequence=[rec, act, dth])
fig.data[0].textinfo = 'label+text+value'
fig.show()

# Total Cumulative COVID-19 Cases in Morocco

In [ ]:
line_data = data.groupby('Date').sum().reset_index()

line_data = line_data.melt(id_vars='Date', 
                 value_vars=['Confirmed', 'Recovered', 'Deaths'], 
                 var_name='Ratio', 
                 value_name='Value')

fig = px.line(line_data, x="Date", y="Value", color='Ratio', 
              title='Total Cumulative COVID-19 Cases in Morocco')
fig.show()

In [ ]:
line_data = data.groupby('Date').sum().reset_index()

line_data = line_data.melt(id_vars='Date', 
                 value_vars=['Active'], 
                 var_name='Ratio', 
                 value_name='Value')

fig = px.line(line_data, x="Date", y="Value", color='Ratio', 
              title='Total Evolution of COVID-19 Active')
fig.show()

# Regional Analysis

In [ ]:
dictionary_labels = {
       'TTH': 'Tanger-Tétouane-Al Hoceima',
       'OR': 'Oriental',
       'FM': 'Fès-Meknès',
       'RSK': 'Rabat-Salé-Kénitra',
       'BK': 'Béni Mellal-Khénifra',
       'CS': 'Casablanca-Settat',
       'MS': 'Marrakech-Safi',
       'DT': 'Drâa-Tafilalet',
       'SM': 'Souss-Massa',
       'GO': 'Guelmim-Oued Noun',
       'LS': 'Laâyoune-Sakia El Hamra',
       'DO': 'Dakhla-Oued Ed-Dahab'
}
data = data.set_index('Date')
regions = []
for x in dictionary_labels:
    regions.append(data[x][-1])

labels = []
for x in dictionary_labels.values():
    labels.append(x)

In [ ]:
plt.figure(figsize=(25,10))
plt.pie(regions, labels = labels)
plt.title('Pie Chart of Total COVID-19 Cases per Region')

In [ ]:
# Treemaps
labels = dictionary_labels.keys()
volume = [data.sum()[x] for x in labels]
color_list = ['#0f7216', '#b2790c', '#ffe9a3',
              '#f9d4d4', '#d35158', '#ea3033',
              '#507b28', '#3e50d3', '#9ffe73',
              '#5989d6', '#e82f4b', '#2b9d80'
             ]

squarify.plot(sizes=volume, label=labels,
              color=color_list, alpha=0.7)

plt.show()

In [ ]:
line_data = data.groupby('Date').sum().reset_index()
line_data = line_data.melt(id_vars='Date', 
                 value_vars=dictionary_labels.keys(), 
                 var_name='Ratio', 
                 value_name='Value')

fig = px.line(line_data, x="Date", y="Value", color='Ratio', 
              title='Total Evolution of COVID-19 Cases per Region')
fig.show()

In [ ]:
labels = list(dictionary_labels.keys())
regions = list(dictionary_labels.values())
plt.figure(figsize=(25,10))
for i in range (12):
       ax1 = plt.subplot(4, 3, i+1)
       sns.lineplot(data = data[labels[i]],label=regions[i])

# Forecasting of Daily Cases
We'll use Facebook developed an open sourcing Prophet as forecasting tool. In fact, Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

In [ ]:
new_cases = []
# data = data.set_index('Date')

for i in range(len(total_Confirmed)):
    if i == 0:
        new_cases.append(0)
    elif total_Confirmed[i] < total_Confirmed[i-1]:
        new_cases.append(0)
    else:
        temp = int(total_Confirmed[i] - total_Confirmed[i-1])
        new_cases.append(temp)
    
new_cases = np.array(new_cases)
data['New cases'] = new_cases
new_cases = data.groupby('Date').sum()['New cases'].reset_index()
new_cases.columns = ['ds','y']

model = Prophet(interval_width = 0.95)
model.fit(new_cases)

future = model.make_future_dataframe(periods = 5)
future.tail()

forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# print("Mean Absolute Error:", mean_absolute_error(forecast['yhat'].head(215).round(), data['New cases']).round())

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast.ds, y = abs(forecast['yhat'].round()),
                         mode= 'lines+markers',name='Forecasting of Daily Cases'))
fig.add_trace(go.Scatter(x=data.index, y = data['New cases'],
                         mode= 'lines+markers',name='Daily cases'))
fig.update_layout(title="Forecasting of COVID-19 Daily Cases",
                 xaxis_title="Date",yaxis_title="Number of Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
plot_plotly(model, forecast)

<h1> Stay Safe & Follow the WHO Guidance!